In [36]:
import numpy as np


In [37]:
def utility_function(candidate, voter):
    """
    Computes the utility as the sum of cube roots of elementwise products.
    
    Parameters:
    -----------
    candidate : array-like
        Vector representing candidate position
    voter : array-like
        Vector representing voter position
    
    Returns:
    --------
    float
        Sum of cube roots of elementwise products
    """
    candidate = np.asarray(candidate)
    voter = np.asarray(voter)
    
    # Elementwise products
    products = candidate * voter
    
    # Cube roots of products
    cube_roots = np.cbrt(products)
    
    # Sum
    return np.sum(cube_roots)

In [38]:
def voting_probability(voter_list, voter_index, candidate_list, candidate_index, p=0.5, default=False):
    """
    Computes the probability of a specific voter voting for a specific candidate.
    
    Parameters:
    -----------
    voter_list : array-like
        List of voter vectors
    voter_index : int
        Index of the specific voter
    candidate_list : array-like
        List of candidate vectors
    candidate_index : int
        Index of the specific candidate
    p : float, default=0.5
        Default probability constant (used when default=True)
    default : bool, default=False
        If True, returns constant probability p.
        If False, uses softmax based on utilities.
    
    Returns:
    --------
    float
        Probability of voter voting for candidate
    """
    if default:
        return p
    
    # Get the specific voter
    voter = np.asarray(voter_list[voter_index])
    
    # Calculate utility for the specific candidate
    utility_i = utility_function(candidate_list[candidate_index], voter)
    
    # Calculate utilities for all candidates
    utilities = []
    for candidate in candidate_list:
        util = utility_function(candidate, voter)
        utilities.append(util)
    
    utilities = np.array(utilities)
    
    # Apply softmax: exp(utility_i) / sum(exp(utilities))
    # Subtract max for numerical stability
    max_util = np.max(utilities)
    exp_utilities = np.exp(utilities - max_util)
    sum_exp = np.sum(exp_utilities)
    
    exp_utility_i = np.exp(utility_i - max_util)
    probability = exp_utility_i / sum_exp
    
    return probability


In [39]:
def generate_voters(n, d, sparsity=0.5, seed=None):
    """
    Generate n voters with true utility vectors and sparse voting vectors.
    
    Parameters:
    -----------
    n : int
        Number of voters
    d : int
        Number of dimensions (policies)
    sparsity : float, default=0.5
        Fraction of dimensions to zero out in voting vector (0 = no sparsity, 1 = all zero)
    seed : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    true_utility_vectors : ndarray
        Shape (n, d) - true utility vectors for each voter (all positive values)
    voting_vectors : ndarray
        Shape (n, d) - sparse voting vectors (some dimensions zeroed)
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Generate true utility vectors (only positive values)
    true_utility_vectors = np.random.uniform(0.1, 10.0, size=(n, d))
    
    # Create sparse voting vectors
    voting_vectors = true_utility_vectors.copy()
    
    # Zero out random dimensions for each voter
    for i in range(n):
        num_zeros = int(d * sparsity)
        zero_indices = np.random.choice(d, size=num_zeros, replace=False)
        voting_vectors[i, zero_indices] = 0
    
    return true_utility_vectors, voting_vectors


In [40]:
def generate_candidates(m, d, budget, seed=None):
    """
    Generate m candidates with policy vectors (budget allocation).
    
    Parameters:
    -----------
    m : int
        Number of candidates
    d : int
        Number of dimensions (policies)
    budget : float
        Total budget to allocate across dimensions
    seed : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    candidate_vectors : ndarray
        Shape (m, d) - policy vectors for each candidate
        Each dimension >= -1, and sum(vector) = budget (full budget is used)
    """
    if seed is not None:
        np.random.seed(seed)
    
    candidate_vectors = np.zeros((m, d))
    
    for i in range(m):
        # Start with all dimensions at -1 (sum = -d)
        # Need to add (budget + d) to get sum = budget
        # Allocate (budget + d) randomly across dimensions
        allocations = np.random.dirichlet(np.ones(d)) * (budget + d)
        
        # Add to -1 baseline: vector = -1 + allocations
        # This ensures: sum(vector) = -d + (budget + d) = budget
        # and each vector[i] >= -1 (since allocations[i] >= 0)
        candidate_vectors[i] = -1 + allocations
    
    # Verify budget constraint
    for i in range(m):
        assert np.all(candidate_vectors[i] >= -1), f"Candidate {i} violates lower bound"
        assert np.abs(np.sum(candidate_vectors[i]) - budget) < 1e-10, \
            f"Candidate {i} doesn't use full budget"
    
    return candidate_vectors


In [41]:
def run_approval_voting_simulation(n, m, d, budget, sparsity=0.5, seed=None):
    """
    Run a complete approval voting simulation using probabilistic voting.
    
    Parameters:
    -----------
    n : int
        Number of voters
    m : int
        Number of candidates
    d : int
        Number of dimensions (policies)
    budget : float
        Budget for each candidate
    sparsity : float, default=0.5
        Sparsity of voting vectors (fraction of dimensions zeroed)
    seed : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    results : dict
        Dictionary containing:
        - 'vote_counts': array of vote counts for each candidate
        - 'true_utility_vectors': voter true utility vectors
        - 'voting_vectors': voter voting vectors
        - 'candidate_vectors': candidate policy vectors
        - 'approval_matrix': (n, m) boolean matrix of approvals
    """
    # Generate voters
    true_utility_vectors, voting_vectors = generate_voters(n, d, sparsity, seed)
    
    # Generate candidates
    candidate_vectors = generate_candidates(m, d, budget, seed)
    
    # Convert to lists for voting_probability function
    voting_vectors_list = [voting_vectors[i] for i in range(n)]
    candidate_vectors_list = [candidate_vectors[i] for i in range(m)]
    
    # Determine approvals using probabilistic voting
    approval_matrix = np.zeros((n, m), dtype=bool)
    
    for voter_idx in range(n):
        for candidate_idx in range(m):
            # Get voting probability using softmax
            prob = voting_probability(
                voting_vectors_list, 
                voter_index=voter_idx,
                candidate_list=candidate_vectors_list,
                candidate_index=candidate_idx,
                default=False
            )
            
            # Toss weighted coin: approve with probability prob
            approval = np.random.rand() < prob
            approval_matrix[voter_idx, candidate_idx] = approval
    
    # Count votes for each candidate
    vote_counts = np.sum(approval_matrix, axis=0)
    
    results = {
        'vote_counts': vote_counts,
        'true_utility_vectors': true_utility_vectors,
        'voting_vectors': voting_vectors,
        'candidate_vectors': candidate_vectors,
        'approval_matrix': approval_matrix
    }
    
    return results


In [42]:
def select_winner_and_compute_global_utility(vote_counts, candidate_vectors, true_utility_vectors, k, seed=None):
    """
    Select winner from top k candidates and compute global utility.
    
    Parameters:
    -----------
    vote_counts : array-like
        Vote counts for each candidate
    candidate_vectors : array-like
        Policy vectors for each candidate
    true_utility_vectors : array-like
        True utility vectors for each voter
    k : int
        Number of top candidates to consider
    seed : int, optional
        Random seed for winner selection
    
    Returns:
    --------
    winner_idx : int
        Index of the winning candidate
    global_utility : float
        Sum of utilities across all voters for the winner
    top_k_indices : array
        Indices of the top k candidates
    """
    if seed is not None:
        np.random.seed(seed)
    
    vote_counts = np.asarray(vote_counts)
    
    # Find top k candidates (handle ties by taking first k)
    # Sort indices by vote count in descending order
    sorted_indices = np.argsort(vote_counts)[::-1]
    top_k_indices = sorted_indices[:k]
    
    # Randomly select winner from top k (uniform probability 1/k)
    winner_idx = np.random.choice(top_k_indices)
    
    # Calculate global utility using true utility vectors
    winner_vector = candidate_vectors[winner_idx]
    global_utility = 0.0
    
    for voter_idx in range(len(true_utility_vectors)):
        true_utility_vector = true_utility_vectors[voter_idx]
        voter_utility = utility_function(winner_vector, true_utility_vector)
        global_utility += voter_utility
    
    return winner_idx, global_utility, top_k_indices


In [43]:
def estimate_expected_global_utility_by_k(n, m, d, budget, sparsity=0.5, n_simulations=100, seed=None):
    """
    Estimate expected global utility for each k (1 to m) using Monte Carlo simulation.
    
    Parameters:
    -----------
    n : int
        Number of voters
    m : int
        Number of candidates
    d : int
        Number of dimensions
    budget : float
        Budget for each candidate
    sparsity : float, default=0.5
        Sparsity of voting vectors
    n_simulations : int, default=100
        Number of Monte Carlo simulations to run
    seed : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    expected_utilities : dict
        Dictionary mapping k -> expected global utility
    std_utilities : dict
        Dictionary mapping k -> standard deviation of global utility
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Store utilities for each k across all simulations
    utilities_by_k = {k: [] for k in range(1, m + 1)}
    
    for sim in range(n_simulations):
        # Run simulation with different seed for each run
        results = run_approval_voting_simulation(
            n=n, m=m, d=d, budget=budget, sparsity=sparsity, seed=None
        )
        
        # For each possible k, compute expected global utility
        vote_counts = results['vote_counts']
        candidate_vectors = results['candidate_vectors']
        true_utility_vectors = results['true_utility_vectors']
        
        # Sort candidates by vote count
        sorted_indices = np.argsort(vote_counts)[::-1]
        
        # For each k, compute expected utility
        for k in range(1, m + 1):
            top_k_indices = sorted_indices[:k]
            
            # Expected utility = average utility of top k candidates
            # (since winner is randomly selected with probability 1/k)
            expected_utility = 0.0
            for candidate_idx in top_k_indices:
                candidate_vector = candidate_vectors[candidate_idx]
                global_utility = 0.0
                
                for voter_idx in range(len(true_utility_vectors)):
                    true_utility_vector = true_utility_vectors[voter_idx]
                    voter_utility = utility_function(candidate_vector, true_utility_vector)
                    global_utility += voter_utility
                
                expected_utility += global_utility / k  # Each has probability 1/k
            
            utilities_by_k[k].append(expected_utility)
    
    # Compute means and standard deviations
    expected_utilities = {k: np.mean(utilities_by_k[k]) for k in range(1, m + 1)}
    std_utilities = {k: np.std(utilities_by_k[k]) for k in range(1, m + 1)}
    
    return expected_utilities, std_utilities


In [44]:
def find_optimal_k(n, m, d, budget, sparsity=0.5, n_simulations=100, seed=None):
    """
    Find the optimal k that maximizes expected global utility.
    
    Parameters:
    -----------
    n : int
        Number of voters
    m : int
        Number of candidates
    d : int
        Number of dimensions
    budget : float
        Budget for each candidate
    sparsity : float, default=0.5
        Sparsity of voting vectors
    n_simulations : int, default=100
        Number of Monte Carlo simulations to run
    seed : int, optional
        Random seed for reproducibility
    
    Returns:
    --------
    optimal_k : int
        The value of k that maximizes expected global utility
    expected_utilities : dict
        Dictionary mapping k -> expected global utility
    std_utilities : dict
        Dictionary mapping k -> standard deviation of global utility
    """
    expected_utilities, std_utilities = estimate_expected_global_utility_by_k(
        n, m, d, budget, sparsity, n_simulations, seed
    )
    
    # Find k with maximum expected utility
    optimal_k = max(expected_utilities, key=expected_utilities.get)
    
    return optimal_k, expected_utilities, std_utilities


In [45]:
# Find optimal k for given parameters
n_voters = 100
m_candidates = 10
d_dimensions = 4
budget = 20.0
sparsity = 0.4
n_simulations = 100  # Increase for more accurate estimates

print("Finding optimal k...")
print("=" * 50)
print(f"Parameters: n={n_voters}, m={m_candidates}, d={d_dimensions}")
print(f"Budget={budget}, Sparsity={sparsity}")
print(f"Running {n_simulations} simulations...\n")

optimal_k, expected_utilities, std_utilities = find_optimal_k(
    n=n_voters,
    m=m_candidates,
    d=d_dimensions,
    budget=budget,
    sparsity=sparsity,
    n_simulations=n_simulations,
    seed=42
)

print(f"Optimal k: {optimal_k}")
print("\nExpected global utility by k:")
for k in range(1, m_candidates + 1):
    marker = " <-- OPTIMAL" if k == optimal_k else ""
    print(f"  k={k}: {expected_utilities[k]:.4f} ± {std_utilities[k]:.4f}{marker}")


Finding optimal k...
Parameters: n=100, m=10, d=4
Budget=20.0, Sparsity=0.4
Running 100 simulations...

Optimal k: 1

Expected global utility by k:
  k=1: 1065.1344 ± 51.9439 <-- OPTIMAL
  k=2: 1055.6630 ± 48.0055
  k=3: 1036.9548 ± 57.2317
  k=4: 1016.5243 ± 58.2890
  k=5: 999.2104 ± 57.0537
  k=6: 982.9076 ± 54.8287
  k=7: 961.9597 ± 53.6914
  k=8: 942.0953 ± 52.0469
  k=9: 918.2783 ± 58.0567
  k=10: 888.0716 ± 60.5471


In [46]:
def run_overfitting_simulation(n_voters, m_candidates, d_dimensions, budget, n_simulations=1000, seed=None, verbose=True):
    """
    Run the overfitting simulation where:
    - Each voter's voting vector is a standard basis vector (one-hot)
    - Each voter's true utility vector has 1 in all dimensions (cares about all)
    - Each candidate overfits to one dimension
    
    Parameters:
    -----------
    n_voters : int
        Number of voters
    m_candidates : int
        Number of candidates
    d_dimensions : int
        Number of dimensions
    budget : float
        Budget for each candidate
    n_simulations : int, default=1000
        Number of simulations for expected utility calculation
    seed : int, optional
        Random seed for reproducibility
    verbose : bool, default=True
        Whether to print detailed output
    
    Returns:
    --------
    results : dict
        Dictionary containing:
        - 'voting_vectors': voter voting vectors
        - 'true_utility_vectors': voter true utility vectors
        - 'candidate_vectors': candidate policy vectors
        - 'expected_utilities': dict mapping k -> expected global utility
        - 'std_utilities': dict mapping k -> std dev of global utility
        - 'optimal_k': optimal k value
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Set up voters: each has a standard basis vector as their voting vector
    voting_vectors = np.zeros((n_voters, d_dimensions))
    for i in range(n_voters):
        voting_vectors[i, i] = 1.0  # Standard basis vector
    
    # True utility vectors: each voter cares about all dimensions equally
    true_utility_vectors = np.ones((n_voters, d_dimensions))
    
    # Set up candidates: each candidate overfits to one dimension
    candidate_vectors = np.full((m_candidates, d_dimensions), -1.0)
    for i in range(m_candidates):
        # Overfitting dimension gets: budget + (d_dimensions - 1)
        candidate_vectors[i, i] = budget + (d_dimensions - 1)
    
    if verbose:
        print("Custom Overfitting Simulation Setup")
        print("=" * 50)
        print(f"Voters: {n_voters}, Candidates: {m_candidates}, Dimensions: {d_dimensions}")
        print(f"Budget: {budget}")
        print("\nVoting vectors (each voter focuses on one dimension - myopic voting):")
        for i in range(n_voters):
            print(f"  Voter {i+1}: {voting_vectors[i]}")
        print("\nTrue utility vectors (each voter cares about all dimensions):")
        for i in range(n_voters):
            print(f"  Voter {i+1}: {true_utility_vectors[i]}")
        print("\nCandidate policy vectors (each overfits to one dimension):")
        for i in range(m_candidates):
            print(f"  Candidate {i+1}: {candidate_vectors[i]}")
            budget_used = np.sum(candidate_vectors[i])
            print(f"    Sum of vector: {budget_used:.2f} (target: {budget:.2f})")
            assert np.abs(budget_used - budget) < 1e-10, f"Candidate {i+1} budget constraint violated"
    
    # Convert to lists for voting_probability function
    voting_vectors_list = [voting_vectors[i] for i in range(n_voters)]
    candidate_vectors_list = [candidate_vectors[i] for i in range(m_candidates)]
    
    # Compute expected global utility for each k
    utilities_by_k = {k: [] for k in range(1, m_candidates + 1)}
    
    for sim in range(n_simulations):
        # Run probabilistic voting
        approval_matrix = np.zeros((n_voters, m_candidates), dtype=bool)
        for voter_idx in range(n_voters):
            for candidate_idx in range(m_candidates):
                prob = voting_probability(
                    voting_vectors_list, 
                    voter_index=voter_idx,
                    candidate_list=candidate_vectors_list,
                    candidate_index=candidate_idx,
                    default=False
                )
                approval = np.random.rand() < prob
                approval_matrix[voter_idx, candidate_idx] = approval
        
        vote_counts = np.sum(approval_matrix, axis=0)
        
        # Sort candidates by vote count
        sorted_indices = np.argsort(vote_counts)[::-1]
        
        # For each k, compute expected utility
        for k in range(1, m_candidates + 1):
            top_k_indices = sorted_indices[:k]
            
            # Expected utility = average utility of top k candidates
            expected_utility = 0.0
            for candidate_idx in top_k_indices:
                candidate_vector = candidate_vectors[candidate_idx]
                global_utility = 0.0
                
                for voter_idx in range(len(true_utility_vectors)):
                    true_utility_vector = true_utility_vectors[voter_idx]
                    voter_utility = utility_function(candidate_vector, true_utility_vector)
                    global_utility += voter_utility
                
                expected_utility += global_utility / k  # Each has probability 1/k
            
            utilities_by_k[k].append(expected_utility)
    
    # Compute means and standard deviations
    expected_utilities = {k: np.mean(utilities_by_k[k]) for k in range(1, m_candidates + 1)}
    std_utilities = {k: np.std(utilities_by_k[k]) for k in range(1, m_candidates + 1)}
    
    # Find optimal k
    optimal_k = max(expected_utilities, key=expected_utilities.get)
    
    if verbose:
        print("\nOptimal k Analysis")
        print("=" * 50)
        print(f"Optimal k: {optimal_k}")
        print("\nExpected global utility by k:")
        for k in range(1, m_candidates + 1):
            marker = " <-- OPTIMAL" if k == optimal_k else ""
            print(f"  k={k}: {expected_utilities[k]:.4f} ± {std_utilities[k]:.4f}{marker}")
    
    results = {
        'voting_vectors': voting_vectors,
        'true_utility_vectors': true_utility_vectors,
        'candidate_vectors': candidate_vectors,
        'expected_utilities': expected_utilities,
        'std_utilities': std_utilities,
        'optimal_k': optimal_k
    }
    
    return results


In [47]:
# Run the overfitting simulation
n_voters = 5
m_candidates = 5
d_dimensions = 6
budget = 5.0
n_simulations = 1000

results = run_overfitting_simulation(
    n_voters=n_voters,
    m_candidates=m_candidates,
    d_dimensions=d_dimensions,
    budget=budget,
    n_simulations=n_simulations,
    seed=42,
    verbose=True
)


Custom Overfitting Simulation Setup
Voters: 5, Candidates: 5, Dimensions: 6
Budget: 5.0

Voting vectors (each voter focuses on one dimension - myopic voting):
  Voter 1: [1. 0. 0. 0. 0. 0.]
  Voter 2: [0. 1. 0. 0. 0. 0.]
  Voter 3: [0. 0. 1. 0. 0. 0.]
  Voter 4: [0. 0. 0. 1. 0. 0.]
  Voter 5: [0. 0. 0. 0. 1. 0.]

True utility vectors (each voter cares about all dimensions):
  Voter 1: [1. 1. 1. 1. 1. 1.]
  Voter 2: [1. 1. 1. 1. 1. 1.]
  Voter 3: [1. 1. 1. 1. 1. 1.]
  Voter 4: [1. 1. 1. 1. 1. 1.]
  Voter 5: [1. 1. 1. 1. 1. 1.]

Candidate policy vectors (each overfits to one dimension):
  Candidate 1: [10. -1. -1. -1. -1. -1.]
    Sum of vector: 5.00 (target: 5.00)
  Candidate 2: [-1. 10. -1. -1. -1. -1.]
    Sum of vector: 5.00 (target: 5.00)
  Candidate 3: [-1. -1. 10. -1. -1. -1.]
    Sum of vector: 5.00 (target: 5.00)
  Candidate 4: [-1. -1. -1. 10. -1. -1.]
    Sum of vector: 5.00 (target: 5.00)
  Candidate 5: [-1. -1. -1. -1. 10. -1.]
    Sum of vector: 5.00 (target: 5.00)

Optimal

In [48]:
# Example simulation
n_voters = 100
m_candidates = 5
d_dimensions = 10
budget = 20.0
sparsity = 0.4  # 40% of dimensions zeroed out in voting vectors
k = 3  # Top k candidates to consider for winner selection

results = run_approval_voting_simulation(
    n=n_voters,
    m=m_candidates,
    d=d_dimensions,
    budget=budget,
    sparsity=sparsity,
    seed=42
)

print("Approval Voting Simulation Results")
print("=" * 50)
print(f"Voters: {n_voters}, Candidates: {m_candidates}, Dimensions: {d_dimensions}")
print(f"Budget per candidate: {budget}, Voting vector sparsity: {sparsity}")
print("\nVote counts for each candidate:")
for i, votes in enumerate(results['vote_counts']):
    print(f"  Candidate {i+1}: {votes} votes ({votes/n_voters*100:.1f}% approval rate)")

print(f"\nTotal votes cast: {np.sum(results['vote_counts'])}")
print(f"Average approvals per voter: {np.sum(results['approval_matrix']) / n_voters:.2f}")

# Select winner from top k and compute global utility
winner_idx, global_utility, top_k_indices = select_winner_and_compute_global_utility(
    results['vote_counts'],
    results['candidate_vectors'],
    results['true_utility_vectors'],
    k=k,
    seed=42
)

print(f"\nTop {k} candidates (by vote count): {[i+1 for i in top_k_indices]}")
print(f"Winner (randomly selected from top {k}): Candidate {winner_idx+1}")
print(f"Global utility (using true utility vectors): {global_utility:.4f}")
print(f"Average utility per voter: {global_utility/n_voters:.4f}")


Approval Voting Simulation Results
Voters: 100, Candidates: 5, Dimensions: 10
Budget per candidate: 20.0, Voting vector sparsity: 0.4

Vote counts for each candidate:
  Candidate 1: 26 votes (26.0% approval rate)
  Candidate 2: 8 votes (8.0% approval rate)
  Candidate 3: 32 votes (32.0% approval rate)
  Candidate 4: 8 votes (8.0% approval rate)
  Candidate 5: 29 votes (29.0% approval rate)

Total votes cast: 103
Average approvals per voter: 1.03

Top 3 candidates (by vote count): [3, 5, 1]
Winner (randomly selected from top 3): Candidate 1
Global utility (using true utility vectors): 1151.2201
Average utility per voter: 11.5122
